<a href="https://colab.research.google.com/github/tcharos/Image-Classification-SVM/blob/main/SignalProcessing_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIDL Assignment 2 – Image Classification (MNIST)

Skeleton notebook for experimentation and implementation.
Fill in the code sections as required. Do **not** include answers here; analysis and results should go into the PDF report.

## Imports and Global Settings

In [ ]:
import os
import requests
import tarfile
import numpy as np

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.datasets import mnist

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import time

import torch
from torchvision import datasets

import matplotlib.pyplot as plt




## Functions

## 1. Load and Inspect the MNIST Data

In [ ]:
# 1. Setup Directory
dataset_dir = './dataset'
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    print(f"Created directory: {dataset_dir}")

# 2. Download Dataset
url = "http://www.di.ens.fr/~lelarge/MNIST.tar.gz"
target_path = os.path.join(dataset_dir, "MNIST.tar.gz")

In [ ]:
if not os.path.exists(target_path):
    print("Downloading dataset...")
    response = requests.get(url, stream=True)
    with open(target_path, "wb") as f:
        f.write(response.raw.read())
    print("Download complete.")

if os.path.exists(target_path):
    print("Extracting files...")
with tarfile.open(target_path, "r:gz") as tar:
    tar.extractall(path=dataset_dir)
print("Extraction complete.")

In [ ]:
train_set = datasets.MNIST(root=dataset_dir, train=True, download=False)
test_set = datasets.MNIST(root=dataset_dir, train=False, download=False)

X_train = train_set.data.numpy()
y_train = train_set.targets.numpy()
X_test = test_set.data.numpy()
y_test = test_set.targets.numpy()

print(f"Training shape: {X_train.shape}") # Should be (60000, 28, 28)
print(f"Testing shape: {X_test.shape}")   # Should be (10000, 28, 28)

## 2. Data Regrouping (Binary Classification)

In [ ]:
# Convert labels to binary classes:
# Class 1: digits 0–4, Class 2: digits 5–9

y_train_binary = np.where(y_train < 5, 0, 1)
y_test_binary = np.where(y_test < 5, 0, 1)

# Verification
print(f"Original unique labels: {np.unique(y_train)}")
print(f"New unique labels: {np.unique(y_train_binary)}")


In [ ]:
# verify the split

indices = np.random.choice(len(X_train), 5, replace=False)

plt.figure(figsize=(12, 4))
for i, idx in enumerate(indices):
    plt.subplot(1, 5, i + 1)
    plt.imshow(X_train[idx], cmap='gray')

    original_label = y_train[idx]
    binary_label = y_train_binary[idx]

    # Determine which class it belongs to for the title
    class_name = "Class 1" if binary_label == 0 else "Class 2"

    plt.title(f"Orig: {original_label}\nNew: {binary_label}\n({class_name})")
    plt.axis('off')

plt.tight_layout()
plt.show()

### Preprocessing

In [ ]:
# Flatten images (28x28 -> 784)
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Normalization: Scale pixel values to the [0, 1] range
X_train_flat = X_train_flat.astype('float32') / 255.0
X_test_flat = X_test_flat.astype('float32') / 255.0


In [ ]:
# sub-sampling the dataset
# pick every 3rd vector and discards the rest
X_train_final = X_train_flat[::3]
y_train_final = y_train_binary[::3]

# test set is the same
X_test_final = X_test_flat
y_test_final = y_test_binary

## 3. SVM Benchmark

### SVM GridSearch - Hyperparameter Tuning

In [ ]:
param_grid = [
    {
        'kernel': ['poly'],
        'C': [0.1, 1, 10],
        'degree': [2, 3, 4, 5]  # Exploring polynomial degree
    },
    {
        'kernel': ['rbf'],
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto', 0.001, 0.01, 0.1] # Exploring Gamma
    },
    {
        'kernel': ['linear'],
        'C': [0.1, 1, 10]
    }
]

In [ ]:
# Grid search over C and kernels (linear, poly, rbf)
# Explore degree (poly) and gamma (rbf)

svc = SVC()

grid_search = GridSearchCV(SVC(),
                           param_grid,
                           cv=3,
                           scoring='accuracy',
                           verbose=1
)

print("Starting Grid Search for SVM Benchmark...")
start_time = time.time()
grid_search.fit(X_train_final, y_train_final)
total_time = time.time() - start_time

print(f"\nGrid Search completed in {total_time:.2f} seconds")
print(f"Best Hyperparameters found: {grid_search.best_params_}")
print(f"Best Cross-Validation Accuracy: {grid_search.best_score_:.4f}")


In [ ]:
best_svm = grid_search.best_estimator_
y_pred = best_svm.predict(X_test_final)
benchmark_accuracy = accuracy_score(y_test_final, y_pred) * 100

print(f"SVM Benchmark Total Accuracy: {benchmark_accuracy:.2f}%")


## 4. Dimensionality reduction via PCA

In [ ]:
# Apply PCA for different K values
# Train SVM on PCA-reduced data
# Store accuracy vs K


### PCA Accuracy Plot

In [ ]:
# Plot SVM accuracy as a function of K


## 5. Dimensionality reduction via LDA

In [ ]:
# Apply LDA (1D output)
# Plot projected test data
# Train and evaluate SVM on LDA features


## 6. Dimensionality reduction via PCA + LDA

In [ ]:
# Apply PCA followed by LDA for different K
# Train SVM and record accuracy


## 7. Naïve Bayes Classifier

In [ ]:
# Train Gaussian Naïve Bayes
# Handle near-zero variance issue (e.g., var smoothing)
# Compare accuracy and computation time with SVM


## 8. Summary (No Answers Here)

In [ ]:
# This section may include brief code comments only.
# All discussion, plots, and conclusions go into the PDF report.
